In [1]:
%cd ../..

/home/eli/AnacondaProjects/combinators


In [2]:
import logging
import torch

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
from combinators import lens, sampler
from combinators.model import collections
from examples.moving_mnist import data
from examples.multi_mnist import apg_inference, generative, proposals

In [5]:
T = 10
NUM_DIGITS = 3
NUM_EPOCHS = 500
NUM_PARTICLES = 5

In [6]:
NUM_HIDDEN_DIGIT = 400
NUM_HIDDEN_LOCATION = 400

In [7]:
DIGIT_SIDE = 28
FRAME_SIDE = 96

WHAT_DIM = 10
WHERE_DIM = 2

In [8]:
data_args = {
    'data': 'movingmnist',
    'batch_size': NUM_PARTICLES,
    'train': True,
    'timesteps': T,
    'num_digits': NUM_DIGITS,
    'frame_size': FRAME_SIDE,
    'dv': 0.1
}

In [ ]:
data_loader, _, _, _ = data.setup_data_loader(**data_args)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
object_codes = generative.InitialObjectCodes(WHAT_DIM)
proposal = proposals.ObjectCodesProposal(_, NUM_HIDDEN_DIGIT, WHAT_DIM)
object_codes = sampler.importance_box('init_what', object_codes, (NUM_PARTICLES,), proposal, lens.PRO(0), lens.PRO(1))

In [ ]:
object_locs  = generative.InitialObjectLocations(WHERE_DIM)
proposal = proposals.InitialLocationsProposal(_, FRAME_SIDE, NUM_HIDDEN_LOCATION, WHERE_DIM)
object_locs = sampler.importance_box('init_where', object_locs, (NUM_PARTICLES,), proposal, lens.PRO(0), lens.PRO(1))

In [ ]:
step_locs = generative.StepObjects(NUM_HIDDEN_DIGIT, WHERE_DIM, WHAT_DIM, _)
proposal = proposals.StepLocationsProposal(_, FRAME_SIDE, NUM_HIDDEN_LOCATION, WHERE_DIM)
step_locs = sampler.importance_box('step_where', step_locs, (NUM_PARTICLES,), proposal, lens.PRO(2), lens.PRO(1)

In [ ]:
step_ssm = collections.parameterize_ssm(lens.PRO(1), lens.PRO(1), step_locs)

In [ ]:
apg_model = (object_codes @ object_locs) >> collections.sequential(step_ssm, T)